In [1]:
import random
import math
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker


In [2]:
def contBatActions2(x,demand,pv_gen,bat_t0,elprices,battery,const1,const2):
    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    lowBound=battery.capacity*0.2
    upBound=battery.capacity*0.8
    dim = len(x)
    bat_action = x[0]-bat_t0
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if charOrDisch == 1:
        if bat_action > 0:
            if x[0] == upBound:
                contChar+=1
                charOrDisch=-1                
        elif bat_action < 0:
            charOrDisch=0
            
    elif charOrDisch == -1:
        if bat_action < 0:
            if x[0] == lowBound:
                contDisch+=1
                charOrDisch=1                
        elif bat_action > 0:
            charOrDisch=0
            
    elif charOrDisch == 0:
        if x[0] == upBound:
            charOrDisch=-1
        elif x[0] == lowBound:
            charOrDisch=1
            
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if charOrDisch == 1:
            if bat_action > 0:
                if x[i] == upBound:
                    contChar+=1
                    charOrDisch=-1                
            elif bat_action < 0:
                charOrDisch=0
            
        elif charOrDisch == -1:
            if bat_action < 0:
                if x[i] == lowBound:
                    contDisch+=1
                    charOrDisch=1                
            elif bat_action > 0:
                charOrDisch=0
            
        elif charOrDisch == 0:
            if x[i] == upBound:
                charOrDisch=-1
            elif x[i] == lowBound:
                charOrDisch=1
        
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
    
    fitness = const1*(grid_sum)-const2*(contChar+contDisch)
    return fitness, charOrDisch, contChar, contDisch, grid_sum

def contBatActions(x,demand,pv_gen,bat_t0,elprices,battery,const1,const2):
    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    lowBound=battery.capacity*0.2
    upBound=battery.capacity*0.8
    if x[0]-lowBound < 0.001:
        x[0]=lowBound
    if upBound-x[0] < 0.001:
        x[0]=upBound
    dim = len(x)
    bat_action = x[0]-bat_t0
    bat_action=float(bat_action)
    if abs(bat_action) <= 0.001:
        bat_action=0.0
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if charOrDisch == 1:
        if bat_action > 0:
            if x[0] == upBound:
                contChar+=1
                charOrDisch=-1                
        elif bat_action < 0:
            charOrDisch=0
            
    elif charOrDisch == -1:
        if bat_action < 0:
            if x[0] == lowBound:
                contDisch+=1
                charOrDisch=1                
        elif bat_action > 0:
            charOrDisch=0
            
    elif charOrDisch == 0:
        if x[0] == upBound:
            charOrDisch=-1
        elif x[0] == lowBound:
            charOrDisch=1
            
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
        bat_action=float(bat_action)
        if abs(bat_action) <= 0.001:
            bat_action=0.0
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if charOrDisch == 1:
            if bat_action > 0:
                if x[i] == upBound:
                    contChar+=1
                    charOrDisch=-1                
            elif bat_action < 0:
                charOrDisch=0
            
        elif charOrDisch == -1:
            if bat_action < 0:
                if x[i] == lowBound:
                    contDisch+=1
                    charOrDisch=1                
            elif bat_action > 0:
                charOrDisch=0
            
        elif charOrDisch == 0:
            if x[i] == upBound:
                charOrDisch=-1
            elif x[i] == lowBound:
                charOrDisch=1
        
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
    
    fitness = const1*(grid_sum)-const2*(contChar+contDisch)
    return fitness, charOrDisch, contChar, contDisch, grid_sum

def costFunctionBatteryLifetime3(x,demand,pv_gen,bat_t0,bat_action_t0,elprices,battery,const1,const2,const3):

    charOrDisch = battery.contAct
    contChar=0
    contDisch=0
    batActionTotal=0
    
    numCharges=0
    numDischarges=0
    batLifeLoss=0
    dischAction=0
    DOD=0
    dischFinish=0
    
    charOrDisch2 = battery.action
    contChar=0
    contDisch=0
    
    numCharChanges=0
    numBatNullState=0
    dim = len(x)
    bat_action = x[0]-bat_t0
    
    if bat_action>5:
        bat_action=5
        x[0]=bat_t0+5
    if bat_action<-5:
        bat_action=-5
        x[0]=bat_t0-5
        
    if bat_action == 0:
        numBatNullState+=1
        
    if charOrDisch2 == 1 and bat_action < 0:
        charOrDisch2=-1
        numCharChanges+=1 
        numDischarges+=1
        DOD=(battery.capacity-x[0])/battery.capacity #vypocet depth of discharge
        dischAction=bat_action
        dischFinish=0
        
    elif charOrDisch2 == -1 and bat_action < 0:
        DOD=(battery.capacity-x[0])/battery.capacity #vypocet depth of discharge
        dischAction=(bat_action_t0+bat_action)
        dischFinish=0
            
    elif charOrDisch2 == -1 and bat_action > 0:
        charOrDisch2=1
        numCharChanges+=1
        numCharges+=1
            
    grid_sum=abs((demand[0]-pv_gen[0]+bat_action))
    batActionTotal+=abs(bat_action)
    for i in range(1,dim):
        bat_action = x[i]-x[i-1]
            
        if bat_action>5:
            bat_action=5
            x[i]=x[i-1]+5
        if bat_action<-5:
            bat_action=-5
            x[i]=x[i-1]-5
            
        if bat_action == 0:
            numBatNullState+=1
            
        if charOrDisch2 == 1 and bat_action < 0:
            charOrDisch2=-1
            numCharChanges+=1 
            numDischarges+=1
            DOD=(battery.capacity-x[i])/battery.capacity #vypocet depth of discharge
            dischAction=bat_action
            dischFinish=0
            
        elif charOrDisch2 == -1 and bat_action < 0:
            DOD=(battery.capacity-x[i])/battery.capacity #vypocet depth of discharge
            dischAction+=(bat_action)
            dischFinish=0
            
        elif charOrDisch2 == -1 and bat_action > 0:
            charOrDisch2=1
            numCharChanges+=1
            numCharges+=1
            batLifeLoss+=(abs(dischAction)*DOD)
            dischFinish=1
            
        
        grid_sum+=abs((demand[i]-pv_gen[i]+bat_action))
        batActionTotal+=abs(bat_action)

    if dischFinish==0:
        batLifeLoss+=(abs(dischAction)*DOD)
    if numDischarges>0:
        batLifeLoss=batLifeLoss#/numDischarges
    else:
        batLifeLoss=0
    fitness = (const1*grid_sum)+(const2*numBatNullState)+(const3*batLifeLoss)
    return fitness, charOrDisch2, numCharges, numDischarges, grid_sum, numCharChanges, batLifeLoss

In [3]:
class Battery:
    def __init__(self,cap,soc,act,contAct,numContChar,numContDisch,numCharges,numDischarges,numBatNullState,numCharChanges):
        self.capacity=cap
        self.SOC=soc
        self.action=act
        self.contAct=contAct
        self.numContChar=numContChar
        self.numContDisch=numContDisch
        self.numCharges=numCharges
        self.numDischarges=numDischarges
        self.numBatNullState=numBatNullState
        self.numCharChanges=numCharChanges
        self.batLifeLoss=0
        self.avgDOD=0
        self.sumDOD=0
        self.avgDODPerDisch=0
        self.sumDODPerDisch=0
        self.avgSOC=0
        self.sumSOC=0
        self.avgDischAction=0
        self.sumDischAction=0
        self.totalBat=0

In [4]:
class FireflyAlgorithm():

    def __init__(self, D, NP, nFES, alpha, betamin, gamma, LB, UB, function, demand,pv_gen,bat_t0,bat_action_t0,elprices,battery,const1,const2,const3):
        self.D = D  # dimension of the problem
        self.NP = NP  # population size
        self.nFES = nFES  # number of function evaluations
        self.alpha = alpha  # alpha parameter
        self.betamin = betamin  # beta parameter
        self.gamma = gamma  # gamma parameter
        # sort of fireflies according to fitness value
        self.Index = [0] * self.NP
        self.Fireflies = [[0 for i in range(self.D)]
                          for j in range(self.NP)]  # firefly agents
        self.Fireflies_tmp = [[0 for i in range(self.D)] for j in range(
            self.NP)]  # intermediate pop
        self.Fitness = [0.0] * self.NP  # fitness values
        self.I = [0.0] * self.NP  # light intensity
        self.nbest = [0.0] * self.NP  # the best solution found so far
        self.LB = LB  # lower bound
        self.UB = UB  # upper bound
        self.fbest = None  # the best
        self.evaluations = 0
        self.Fun = function
        
        self.pvToDemand=0           #pridal
        self.battery = battery      #pridal
        
        self.charOrDischBest = 0   #pridal
        self.contCharBest = 0      #pridal
        self.contDischBest = 0     #pridal
        self.gridPriceBest = 0     #pridal
        
        self.charOrDisch2Best = 0      #pridal
        self.numBatNullStateBest = 0   #pridal
        self.numCharChangesBest = 0    #pridal
        self.numChargesBest = 0        #pridal
        self.numDischargesBest = 0     #pridal
        
        self.charOrDisch2 = [0] * self.NP      #pridal
        self.numBatNullState = [0] * self.NP   #pridal
        self.numCharChanges = [0] * self.NP    #pridal
        self.numCharges = [0] * self.NP        #pridal
        self.numDischarges = [0] * self.NP     #pridal
        
        self.charOrDisch = [0] * self.NP #pridal
        self.contChar = [0] * self.NP    #pridal
        self.contDisch = [0] * self.NP   #pridal
        self.gridPrice = [0] * self.NP   #pridal
        self.const1 = const1      #pridal
        self.const2 = const2      #pridal
        self.const3 = const3      #pridal
        
        self.best = [0] * self.D   #pridal
        self.demand = demand       #pridal
        self.pv_gen = pv_gen       #pridal
        self.elprices = elprices   #pridal
        self.bat_t0 = bat_t0       #pridal
        self.bat_action_t0 = bat_action_t0 #pridal
        self.grid_actions = []     #pridal
        self.demand_sum = 0        #pridal
        self.grid_total = 0        #pridal

    def init_ffa(self):
        for i in range(self.NP):
            for j in range(self.D):
                self.Fireflies[i][j] = random.uniform(
                    0, 1) * (self.UB - self.LB) + self.LB
            self.Fitness[i] = 1.0  # initialize attractiveness
            self.I[i] = self.Fitness[i]

    def alpha_new(self, a):
        delta = 1.0 - math.pow((math.pow(10.0, -4.0) / 0.9), 1.0 / float(a))
        return (1 - delta) * self.alpha

    def sort_ffa(self):  # implementation of bubble sort
        for i in range(self.NP):
            self.Index[i] = i

        for i in range(0, (self.NP - 1)):
            j = i + 1
            for j in range(j, self.NP):
                if (self.I[i] > self.I[j]):
                    z = self.I[i]  # exchange attractiveness
                    self.I[i] = self.I[j]
                    self.I[j] = z
                    z = self.Fitness[i]  # exchange fitness
                    self.Fitness[i] = self.Fitness[j]
                    self.Fitness[j] = z
                    z = self.Index[i]  # exchange indexes
                    self.Index[i] = self.Index[j]
                    self.Index[j] = z
                    
                    #pridane pre Bat lifetime
                    z = self.charOrDisch[i]  # exchange battery lifetime
                    self.charOrDisch[i] = self.charOrDisch[j]
                    self.charOrDisch[j] = z
                    z = self.contChar[i]  # exchange battery lifetime
                    self.contChar[i] = self.contChar[j]
                    self.contChar[j] = z
                    z = self.contDisch[i]  # exchange battery lifetime
                    self.contDisch[i] = self.contDisch[j]
                    self.contDisch[j] = z
                    z = self.gridPrice[i]  # exchange battery lifetime
                    self.gridPrice[i] = self.gridPrice[j]
                    self.gridPrice[j] = z
                    
                    z = self.charOrDisch2[i]  # exchange battery lifetime
                    self.charOrDisch2[i] = self.charOrDisch2[j]
                    self.charOrDisch2[j] = z
                    z = self.numCharChanges[i]  # exchange battery lifetime
                    self.numCharChanges[i] = self.numCharChanges[j]
                    self.numCharChanges[j] = z
                    z = self.numBatNullState[i]  # exchange battery lifetime
                    self.numBatNullState[i] = self.numBatNullState[j]
                    self.numBatNullState[j] = z
                    z = self.numCharges[i]  # exchange battery lifetime
                    self.numCharges[i] = self.numCharges[j]
                    self.numCharges[j] = z
                    z = self.numDischarges[i]  # exchange battery lifetime
                    self.numDischarges[i] = self.numDischarges[j]
                    self.numDischarges[j] = z
                    #pridane pre Bat lifetime

    def replace_ffa(self):  # replace the old population according to the new Index values
        # copy original population to a temporary area
        for i in range(self.NP):
            for j in range(self.D):
                self.Fireflies_tmp[i][j] = self.Fireflies[i][j]

        # generational selection in the sense of an EA
        for i in range(self.NP):
            for j in range(self.D):
                self.Fireflies[i][j] = self.Fireflies_tmp[self.Index[i]][j]

    def FindLimits(self, k):
        for i in range(self.D):
            if self.Fireflies[k][i] < self.LB:
                self.Fireflies[k][i] = self.LB
            if self.Fireflies[k][i] > self.UB:
                self.Fireflies[k][i] = self.UB

    def move_ffa(self):
        for i in range(self.NP):
            scale = abs(self.UB - self.LB)
            for j in range(self.NP):
                r = 0.0
                for k in range(self.D):
                    r += (self.Fireflies[i][k] - self.Fireflies[j][k]) * \
                        (self.Fireflies[i][k] - self.Fireflies[j][k])
                r = math.sqrt(r)
                if self.I[i] > self.I[j]:  # brighter and more attractive
                    beta0 = 1.0
                    beta = (beta0 - self.betamin) * \
                        math.exp(-self.gamma * math.pow(r, 2.0)) + self.betamin
                    for k in range(self.D):
                        r = random.uniform(0, 1)
                        tmpf = self.alpha * (r - 0.5) * scale
                        self.Fireflies[i][k] = self.Fireflies[i][
                            k] * (1.0 - beta) + self.Fireflies_tmp[j][k] * beta + tmpf
            self.FindLimits(i)

    def Run(self):
        self.init_ffa()
        
        while self.evaluations < self.nFES:

            # optional reducing of alpha
            self.alpha = self.alpha_new(self.nFES/self.NP)

            # evaluate new solutions
            for i in range(self.NP):
                self.Fitness[i], self.charOrDisch2[i], self.numCharges[i], self.numDischarges[i], self.gridPrice[i], self.numCharChanges[i], self.numBatNullState[i] = self.Fun(self.Fireflies[i],self.demand,self.pv_gen,self.bat_t0,self.bat_action_t0,self.elprices,self.battery,self.const1,self.const2,self.const3)
                #self.Fitness[i], self.charOrDisch[i], self.contChar[i], self.contDisch[i], self.gridPrice[i] = self.Fun(self.Fireflies[i],self.demand,self.pv_gen,self.bat_t0,self.elprices,self.battery,self.const1,self.const2)
                self.evaluations = self.evaluations + 1
                self.I[i] = self.Fitness[i]

            # ranking fireflies by their light intensity
            self.sort_ffa()
            # replace old population
            self.replace_ffa()
            # find the current best
            self.fbest = self.I[0]
            
            #pridane pre Bat lifetime
            self.charOrDischBest = self.charOrDisch[0]
            self.contCharBest = self.contChar[0]
            self.contDischBest = self.contDisch[0]
            self.gridPriceBest = self.gridPrice[0]
            self.best = self.Fireflies[0]
            
            self.charOrDisch2Best = self.charOrDisch2[0]
            self.numBatNullStateBest = self.numBatNullState[0]
            self.numCharChangesBest = self.numCharChanges[0]
            self.numChargesBest =  self.numCharges[0]
            self.numDischargesBest =  self.numDischarges[0]
            
            self.move_ffa()
        
        return self.fbest

In [6]:
class FAOptimizer():
    def __init__(self, Nparticles, Niter, start, end, windowSize,const1,const2,const3, batCapacity, batSOC, houseNumber):
        global str
        self.Nparticles=Nparticles
        self.Niter=Niter
        self.houseNumber=houseNumber
        self.totalImport = 0
        self.totalExport = 0
        self.start = start
        self.end = end
        self.windowSize=windowSize
        self.bat=Battery(cap=batCapacity, soc=batSOC, act=1, contAct=0, numContChar=0, numContDisch=0, numCharges=1, numDischarges=0, numBatNullState=0, numCharChanges=0)
        self.const1=const1
        self.const2=const2
        self.const3=const3
        
        #polia nakladania s energiou
        self.gridTotalProgress = []
        self.pvToDemandProgress = []
        self.batSOCProgress = []
        self.batActionsProgress = []
        
        self.totalPrice=0
        self.total_demand=0
        self.pvToDemandTotal=0
        
        self.data = pd.read_csv('../data/load/dataid' + str(houseNumber) + '.csv', index_col=0, sep=";")
        self.data.loc[self.data['loadProd'] < 0, 'loadProd'] = 0
        
        self.elprices = pd.read_csv('../data/elprices.csv', index_col=0, sep=",")
        self.elprices = pd.DataFrame(self.elprices[['prices']])
        self.elprices = self.elprices['prices']

        self.iter=0
        self.gen = pd.DataFrame(self.data[['loadProd']])
        self.con = pd.DataFrame(self.data[['loadCons']])
        self.date = pd.DataFrame(self.data[['date']])
        self.dayOfWeek = pd.DataFrame(self.data[['dayOfWeek']])
        self.con = self.con['loadCons']
        self.gen = self.gen['loadProd']
   
        
    def calculateImportExport(self,gridAction):
        if gridAction >= 0:
            self.totalImport+=gridAction
        if gridAction < 0:
            self.totalExport+=(-gridAction)
            
    def calculatePrice(self,elprices,gridActions):
        price=0
        for i in range(0,len(elprices)):
            if gridActions[i]>0:
                price+=(elprices[i]*gridActions[i])
        return price
    
    def saveResults(self,gridTotalProgress,batSOCProgress,batActionsProgress,pvToDemandProgress,total_demand, pvToDemandTotal,contAct,numContChar,numContDisch,totalPrice,finalnumCharges,finalnumDischarges,finalnumCharChanges,finalnumNullStates):    
        df=self.data[self.start-1:self.end]
        batSoc = batSOCProgress
        if self.iter==0:
            batSoc.pop(0)
            self.iter=1
        dat1 = pd.DataFrame({'newLoadGrid': gridTotalProgress})
        dat2 = pd.DataFrame({'batterySOC': batSoc})
        dat3 = pd.DataFrame({'batteryActions': batActionsProgress})
        dat4 = pd.DataFrame({'pvUsage': pvToDemandProgress})
        dat5 = pd.DataFrame({'prices': self.elprices[self.start-1:self.end]})
        dat5.index = np.arange(0, len(dat5) + 0)
        dat6 = pd.DataFrame({'batteryLastSOC': [self.bat.SOC]})
        dat7 = pd.DataFrame({'totalPrice': [totalPrice]})
        dat8 = pd.DataFrame({'totalImport': [self.totalImport]})
        dat9 = pd.DataFrame({'totalExport': [self.totalExport]})
        dat10 = pd.DataFrame({'totalCons': [total_demand]})
        dat11 = pd.DataFrame({'totalPvUsage': [pvToDemandTotal]})
        dat12 = pd.DataFrame({'contAct': [contAct]})
        dat26 = pd.DataFrame({'batteryAct': [self.bat.action]})
        dat13 = pd.DataFrame({'numContChar': [numContChar]})
        dat14 = pd.DataFrame({'numContDisch': [numContDisch]})
        
        dat15 = pd.DataFrame({'charges': [finalnumCharges]})
        dat16 = pd.DataFrame({'discharges': [finalnumDischarges]})
        dat17 = pd.DataFrame({'CharChanges': [finalnumCharChanges]})
        dat18 = pd.DataFrame({'batNullStates': [finalnumNullStates]})
        
        dat19 = pd.DataFrame({'batLifeLoss': [self.bat.batLifeLoss]})
        if finalnumDischarges==0:
            dat20 = pd.DataFrame({'batLifeLossPerDisch': [0]})
        else:
            dat20 = pd.DataFrame({'batLifeLossPerDisch': [self.bat.batLifeLoss/finalnumDischarges]})
        dat21 = pd.DataFrame({'avgDOD': [self.bat.avgDOD]})
        dat22 = pd.DataFrame({'avgDODPerDisch': [self.bat.avgDODPerDisch]})
        dat23 = pd.DataFrame({'avgDischAction': [self.bat.avgDischAction]})
        dat24 = pd.DataFrame({'DischAction': [self.bat.avgDischAction*finalnumDischarges]})
        dat25 = pd.DataFrame({'avgSOC': [self.bat.avgSOC]})
        dat27 = pd.DataFrame({'totalBat': [self.bat.totalBat]})
        
        dat28 = pd.DataFrame({'sumDOD': [self.bat.sumDOD]})
        dat29 = pd.DataFrame({'sumDODPerDisch': [self.bat.sumDODPerDisch]})
        dat30 = pd.DataFrame({'sumSOC': [self.bat.sumSOC]})
        dat31 = pd.DataFrame({'sumDischAction': [self.bat.sumDischAction]})
        
        dat1 = pd.concat([dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8, dat9, dat10,dat11, dat12, dat26, dat13, dat14, dat15, dat16, dat17, dat18, dat19, dat20, dat21, dat22, dat23, dat24, dat25, dat27, dat28, dat29, dat30, dat31], axis=1)
        dat1.index = np.arange(self.start, len(dat1) + self.start)
        df = pd.concat([df, dat1], axis=1)
        df.to_csv("Vysledky/FAMaxBatLife/FAMaxBatLifeData"+str(self.houseNumber)+".csv",sep=";")
        

    def calculateContCharDisch(self,batSOCProgress,batSoc,bat):
        conWindow=[]
        genWindow=[]
        elprices=[]
        for i in range(self.start,self.end+1):
            conWindow.append(self.con[i])
            genWindow.append(self.gen[i])
            elprices.append(self.elprices[i])
        vvar, vcontAct, vnumContChar,vnumContDisch, vvar3 = contBatActions(batSOCProgress,conWindow,genWindow,batSoc,elprices,bat,self.const1, self.const2)
        self.bat.numContChar = vnumContChar
        self.bat.numContDisch = vnumContDisch
        print('cont actions ',vcontAct, vnumContChar,vnumContDisch)
        
    def calculateBatteryChargingActions(self, Algorithm, conWindow, genWindow, elprices):
        #vypocet kontinualnych nabiti
        var1, charOrDisch, numContChar, numContDisch, var3 = contBatActions2(Algorithm.best,conWindow, genWindow, self.bat.SOC, elprices,self.bat,self.const1, self.const2)
        self.bat.contAct=charOrDisch

        self.bat.action=Algorithm.charOrDisch2Best
        self.bat.numCharChanges+=Algorithm.numCharChangesBest   
        self.bat.numCharges+=Algorithm.numChargesBest
        self.bat.numDischarges+=Algorithm.numDischargesBest

    
    def calculateBatteryLossActions(self,batActionsProgress,batSOCProgress,charOrDisch):
        numCharChanges = 0
        numCharges = 1
        numDischarges = 0
        numNullStates=0
        batLifeLoss=0
        dischAction=0
        dischFinish=0
        DODProgress=[]
        DODProgress2=[]
        DOD=0
        avgDOD=0
        avgDOD2=0
        avgSOC=0
        avgDischAction=0
        length = len(batActionsProgress)
        for i in range(0,length):
            batAction=batActionsProgress[i]
            avgSOC+=(batSOCProgress[i]/self.bat.capacity)
            avgDOD2+=(self.bat.capacity-batSOCProgress[i])
            DODProgress2.append((self.bat.capacity-batSOCProgress[i]))
            
            if charOrDisch==1 and batAction < 0:
                numCharChanges+=1
                numDischarges+=1
                charOrDisch=-1
                DOD=(self.bat.capacity-batSOCProgress[i])/self.bat.capacity #vypocet depth of discharge
                dischAction=batAction
                dischFinish=0
                
            elif charOrDisch == -1 and batAction < 0:
                DOD=(self.bat.capacity-batSOCProgress[i])/self.bat.capacity #vypocet depth of discharge
                dischAction+=(batAction)
                dischFinish=0
                
            elif charOrDisch==-1 and batAction > 0:
                numCharChanges+=1
                numCharges+=1
                charOrDisch=1
                batLifeLoss+=(abs(dischAction)*DOD)
                avgDischAction+=abs(dischAction)
                avgDOD+=DOD
                DODProgress.append(DOD)
                dischFinish=1
                
            elif batAction == 0:
                numNullStates+=1
                
        if dischFinish==0:
            batLifeLoss+=(abs(dischAction)*DOD)
            avgDOD+=DOD
            DODProgress.append(DOD)
        if numDischarges>0:
            batLifeLoss=batLifeLoss/numDischarges
            avgDOD=avgDOD/numDischarges
            avgDischAction=avgDischAction/numDischarges
        else:
            batLifeLoss=0
        print("numCharChanges ",numCharChanges)
        print("numCharges ",numCharges)
        print("numDischarges ",numDischarges)
        print("numNullStates ",numNullStates)
        self.bat.numBatNullState=numNullStates
        print("batLifeLoss ",batLifeLoss*numDischarges)
        #print("avgDOD ",(sum(DODProgress)/len(DODProgress)),len(DODProgress))
        print("avgDOD ",(sum(DODProgress2)/len(DODProgress2))/self.bat.capacity)
        print("avgSOC ",(sum(batSOCProgress)/len(batSOCProgress))/self.bat.capacity)
        print("avgDischAction ",avgDischAction*numDischarges)
        
        totalBat=0
        for i in batActionsProgress:
            totalBat+=abs(i)
        print('totalBat: ',totalBat)
        
        self.bat.batLifeLoss=batLifeLoss*numDischarges
        self.bat.avgDOD=(sum(DODProgress2)/len(DODProgress2))/self.bat.capacity
        self.bat.avgDODPerDisch=(sum(DODProgress)/len(DODProgress))
        self.bat.avgSOC=(sum(batSOCProgress)/len(batSOCProgress))/self.bat.capacity
        self.bat.avgDischAction=avgDischAction
        self.bat.totalBat=totalBat
        
        self.bat.sumDOD=sum(DODProgress2)
        self.bat.sumDODPerDisch=sum(DODProgress)
        self.bat.sumSOC=sum(batSOCProgress)
        self.bat.sumDischAction=avgDischAction*numDischarges
    
    def optimize(self):
        print("OPTIMALIZACIA particles x iter: ",self.Nparticles,self.Niter)
        windowSize=self.windowSize
        start=self.start#2162#3626
        end = self.end#24#8761#2172#3700#761#4345
        
        total_grid = 0
        total_price = 0
        batteryAction=0
        
        self.batSOCProgress.append(0)
        import time
        zac = int(round(time.time() * 1000))
        print(zac)
        for c in range(start,end,windowSize):
            d=c+windowSize
            initial= []
            genWindow = []
            conWindow = []
            BSSBounds = []
            elprices = []
            print("row number: ",c)
            print("con: ",self.con[c])
            for i in range(c,d):
                if i<=end:
                    elprices.append(self.elprices[i])
                    genWindow.append(self.gen[i])
                    conWindow.append(self.con[i])
                    BSSBounds.append((0,self.bat.capacity))
  
            Algorithm = FireflyAlgorithm(len(genWindow), self.Nparticles, self.Niter, 0.5, 0.2, 1.0,self.bat.capacity*0.2,self.bat.capacity*0.8, costFunctionBatteryLifetime3, conWindow, genWindow, self.bat.SOC, batteryAction, elprices, self.bat, self.const1, self.const2, self.const3)
            Best = Algorithm.Run()
            
            #prerobene
            alg_bat_actions = []                                                
            alg_grid_actions = []
            alg_bat_actions.append(Algorithm.best[0]-self.bat.SOC)
            batteryAction=Algorithm.best[0]-self.bat.SOC
            self.batActionsProgress.append(batteryAction)
            gridActionHour = (conWindow[0]-genWindow[0]+alg_bat_actions[0])
            alg_grid_actions.append(gridActionHour)
            self.calculateImportExport(gridActionHour)
            alg_demand_sum = conWindow[0]
            alg_grid_total = alg_grid_actions[0]
            for k in range(1,Algorithm.D):
                alg_bat_actions.append(Algorithm.best[k]-Algorithm.best[k-1])
                self.batActionsProgress.append(Algorithm.best[k]-Algorithm.best[k-1])
                gridActionHour=(conWindow[k]-genWindow[k]+alg_bat_actions[k])
                alg_grid_actions.append(gridActionHour)
                self.calculateImportExport(gridActionHour)
                alg_demand_sum+=conWindow[k]
                alg_grid_total+=alg_grid_actions[k]
            #prerobene
            
            #vypocet spotreby z pv   
            algPvToDemand=0
            for i in range(0,Algorithm.D):
                pvRest=0
                algPvToDemandInHour = 0
                if genWindow[i] >= conWindow[i]:
                    algPvToDemand += conWindow[i]
                    algPvToDemandInHour += conWindow[i]
                    pvRest = genWindow[i]-conWindow[i]
                else:
                    algPvToDemand += genWindow[i]
                    algPvToDemandInHour += genWindow[i]

                if pvRest > 0 and alg_bat_actions[i]>0:
                    if alg_bat_actions[i]>=pvRest:
                        algPvToDemand += pvRest
                        algPvToDemandInHour += pvRest
                    else:
                        algPvToDemand += alg_bat_actions[i]
                        algPvToDemandInHour += alg_bat_actions[i]
                self.pvToDemandProgress.append(algPvToDemandInHour)
            #vypocet spotreby z pv
            
            self.bat.SOC=Algorithm.best[Algorithm.D-1]
            l = len(alg_grid_actions)
            for j in range(0,l):
                self.gridTotalProgress.append(alg_grid_actions[j])
                self.batSOCProgress.append(Algorithm.best[j])
            self.total_demand += alg_demand_sum#prerobene
            total_grid += alg_grid_total#prerobene
            price = self.calculatePrice(elprices,alg_grid_actions)
            self.totalPrice += price
            
            self.pvToDemandTotal+=algPvToDemand
            
            print()
            
            self.calculateBatteryChargingActions(Algorithm, conWindow, genWindow, elprices)
            self.calculateContCharDisch(self.batSOCProgress,self.bat.SOC,self.bat)
            self.calculateBatteryLossActions(self.batActionsProgress,self.batSOCProgress,1)
            print("row success: ",c)
            
            self.saveResults(self.gridTotalProgress, self.batSOCProgress, self.batActionsProgress, self.pvToDemandProgress,self.total_demand,self.pvToDemandTotal,self.bat.contAct,self.bat.numContChar,self.bat.numContDisch,self.totalPrice,self.bat.numCharges,self.bat.numDischarges,self.bat.numCharChanges,self.bat.numBatNullState)
            
        print()
        print('FINAL:')
        kon = int(round(time.time() * 1000))
        #print((kon/1000)/60)
        print("minut: ",((kon-zac)/1000)/60)
        print('Total demand: ',self.total_demand)
        print('Total price: ',self.totalPrice)
        print('Total Import: ',self.totalImport)
        print('Total Export: ',self.totalExport)
        print('pv to demand: ',self.pvToDemandTotal)
        print()
        
        print()
        print("Charging cycles")
        #self.calculateContCharDisch(self.batSOCProgress,self.bat.SOC,self.bat)
        
        self.calculateBatteryLossActions(self.batActionsProgress,self.batSOCProgress,1)

In [7]:
fa = FAOptimizer(Nparticles=15,Niter=250000,start=1,end=168,windowSize=24,const1=1 ,const2=0, const3=2,batCapacity=13,batSOC=0,houseNumber=744)
fa.optimize()



OPTIMALIZACIA particles x iter:  15 250000
1590878479137
row number:  1
con:  0.4226333333333329

cont actions  -1 1 0
numCharChanges  5
numCharges  3
numDischarges  3
numNullStates  4
batLifeLoss  0.821658861090651
avgDOD  0.5212618428353426
avgSOC  0.48357939675402356
avgDischAction  1.6717028180950444
totalBat:  13.003190203621417
row success:  1
row number:  25
con:  0.38818333333333294

cont actions  -1 1 0
numCharChanges  9
numCharges  5
numDischarges  5
numNullStates  4
batLifeLoss  4.4118112319534974
avgDOD  0.4569459049242414
avgSOC  0.5430540950757586
avgDischAction  7.523658299974345
totalBat:  25.153820350283553
row success:  25
row number:  49
con:  0.6006

cont actions  -1 1 0
numCharChanges  17
numCharges  9
numDischarges  9
numNullStates  4
batLifeLoss  7.445907265113149
avgDOD  0.43690761571065195
avgSOC  0.5630923842893479
avgDischAction  13.665107899614448
totalBat:  37.05958420003496
row success:  49
row number:  73
con:  0.38185

cont actions  -1 1 0
numCharChanges